In [ ]:
from flask import Flask, request
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

app = Flask(__name__)

datagen = ImageDataGenerator(rescale=1./255)
train_data = datagen.flow_from_directory(
    "dataset/train",
    target_size=(64,64),
    batch_size=32,
    class_mode="binary"
)

model = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3)),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, epochs=5)

# Graph
plt.plot(history.history['loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("CNN Training Loss")
plt.show()

@app.route('/')
def home():
    return '''
    <form action="/predict" method="post" enctype="multipart/form-data">
        <input type="file" name="image">
        <input type="submit">
    </form>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    img = request.files['image']
    img.save("img.jpg")
    img = load_img("img.jpg", target_size=(64,64))
    img = img_to_array(img)/255.0
    img = np.expand_dims(img, axis=0)
    result = model.predict(img)
    return "Class 1" if result[0][0] > 0.5 else "Class 0"

if __name__ == '__main__':
    app.run(debug=True)
